---
format: 
  revealjs:
    theme: "solarized" #["theme/q-theme.scss"]
    slide-number: c/t
    logo: "eclipse_logo_small.png"
    footer: "[WS24_DataScienceForEM](https://github.com/ECLIPSE-Lab/WS24_DataScienceForEM)"
    code-copy: true
    center-title-slide: false
    include-in-header: ../heading-meta.html
    code-link: true
    code-overflow: wrap
    highlight-style: a11y
    height: 1080
    width: 1920
    ## output-file: 02_preliminaries.html
execute: 
  eval: true
  echo: true
---

## Convolutional Neural Networks 1
<br>
<h2> Data Science in Electron Microscopy </h2>

<hr>

<h3> Philipp Pelz </h3>

<h3> 2024 </h3>
<br>

<h3>  &nbsp; [https://github.com/ECLIPSE-Lab/WS24_DataScienceForEM](https://github.com/ECLIPSE-Lab/WS24_DataScienceForEM)
</h3>

## Intro 
- small tour of modern CNN architectures. 
- idea of deep neural networks is quite simple (stack together a bunch of layers), performance can vary wildly across architectures and hyperparameter choices. 
- The neural networks described in this chapter are the product of intuition, a few mathematical insights, and a lot of trial and error. 
- batch normalization and residual connections described in this chapter have offered two popular ideas for training and designing deep models, both of which have since also been applied to architectures beyond computer vision.

- AlexNet (Krizhevsky et al., 2012), the first large-scale network deployed to beat conventional computer vision methods on a large-scale vision challenge; 
- the VGG network (Simonyan and Zisserman, 2014), which makes use of a number of repeating blocks of elements; 
- the network in network (NiN) that convolves whole neural networks patch-wise over inputs (Lin et al., 2013); 
- DenseNet (Huang et al., 2017) for a generalization of the residual architecture. 

## Deep Convolutional Neural Networks (AlexNet)

- Pre-CNNs classical pipelines looked more like this:

1. Obtain an interesting dataset. In the early days, these datasets required expensive sensors. For instance, the [Apple QuickTake 100](https://en.wikipedia.org/wiki/Apple_QuickTake) of 1994 sported a whopping 0.3 Megapixel (VGA) resolution, capable of storing up to 8 images, all for the price of \$1,000.
2. Preprocess the dataset with hand-crafted features based on some knowledge of optics, geometry, other analytic tools, and occasionally on the serendipitous discoveries of lucky graduate students.
3. Feed the data through a standard set of feature extractors such as the SIFT (scale-invariant feature transform) :cite:`Lowe.2004`, the SURF (speeded up robust features) :cite:`Bay.Tuytelaars.Van-Gool.2006`, or any number of other hand-tuned pipelines. OpenCV still provides SIFT extractors to this day! 
4. Dump the resulting representations into your favorite classifier, likely a linear model or kernel method, to train a classifier.


In [ ]:
from d2l import torch as d2l
import torch
from torch import nn

## Representation Learning

- Another way to cast the state of affairs is that the most important part of the pipeline was the representation. 
- And up until 2012 the representation was calculated mostly mechanically. In fact, engineering a new set of feature functions, improving results, and writing up the method was a prominent genre of paper. 
- SIFT :cite:`Lowe.2004`, SURF :cite:`Bay.Tuytelaars.Van-Gool.2006`, HOG (histograms of oriented gradient) :cite:`Dalal.Triggs.2005`, bags of visual words :cite:`Sivic.Zisserman.003`, and similar feature extractors ruled the roost. 
- Another group of researchers, including Yann LeCun, Geoff Hinton, Yoshua Bengio, Andrew Ng, Shun-ichi Amari, and Juergen Schmidhuber, had different plans. 
- They believed that features themselves ought to be learned. Moreover, they believed that to be reasonably complex, the features ought to be hierarchically composed with multiple jointly learned layers, each with learnable parameters.

---


- In the case of an image, the lowest layers might come to detect edges, colors, and textures, in analogy to how the visual system in animals processes its input. In particular, the automatic design of visual features such as those obtained by sparse coding :cite:`olshausen1996emergence` remained an open challenge until the advent of modern CNNs. 
- It was not until :citet:`dean2012large,le2013building` that the idea of generating features from image data automatically gained significant traction.
- The first modern CNN :cite:`Krizhevsky.Sutskever.Hinton.2012`, named *AlexNet* after one of its inventors, Alex Krizhevsky, is largely an evolutionary improvement over LeNet. It achieved excellent performance in the 2012 ImageNet challenge.

![Image filters learned by the first layer of AlexNet. Reproduction courtesy of :citet:`Krizhevsky.Sutskever.Hinton.2012`.](../img/filters.png)


In [ ]:
from d2l import torch as d2l
import torch
from torch import nn

## Networks Using Blocks (VGG)

- While AlexNet proved deep CNNs could achieve good results, it didn't provide a template for network design.
- Neural network design has evolved from individual neurons to whole layers, and now to blocks of repeating patterns, similar to how VLSI design progressed from transistors to logic blocks :cite:`Mead.1980`.
- The VGG network from Oxford University pioneered the use of repeated block structures :cite:`Simonyan.Zisserman.2014`, a concept that has evolved into today's foundation models :cite:`bommasani2021opportunities`.



## VGG Blocks

- The basic building block of CNNs consists of:
  - (i) a convolutional layer with padding to maintain resolution
  - (ii) a nonlinearity (ReLU)
  - (iii) a pooling layer to reduce resolution

- Key innovation of VGG: use *multiple* 3×3 convolutions between pooling layers
  - Two 3×3 convolutions = same receptive field as one 5×5
  - Three 3×3 convolutions ($3 \cdot 9 \cdot c^2$ parameters) ≈ one 5×5 ($25 \cdot c^2$ parameters)
  - Deep and narrow networks outperformed shallow ones


In [ ]:
def vgg_block(num_convs, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.LazyConv2d(out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
    layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return nn.Sequential(*layers)

## VGG Network

- VGG Network consists of two parts: 
  - first consisting mostly of convolutional and pooling layers 
  - second consisting of fully connected layers that are identical to those in AlexNet. 
-  key difference : convolutional layers are grouped in nonlinear transformations that leave the dimensonality unchanged, followed by a resolution-reduction step, as depicted in :numref:`fig_vgg`. 

![From AlexNet to VGG. The key difference is that VGG consists of blocks of layers, whereas AlexNet's layers are all designed individually.](../img/vgg.svg){centered=true width=40%}

---


In [ ]:
class VGG(d2l.Classifier):
    def __init__(self, arch, lr=0.1, num_classes=10):
        super().__init__()
        self.save_hyperparameters()
        conv_blks = []
        for (num_convs, out_channels) in arch:
            conv_blks.append(vgg_block(num_convs, out_channels))
        self.net = nn.Sequential(
            *conv_blks, nn.Flatten(),
            nn.LazyLinear(4096), nn.ReLU(), nn.Dropout(0.5),
            nn.LazyLinear(4096), nn.ReLU(), nn.Dropout(0.5),
            nn.LazyLinear(num_classes))
        self.net.apply(d2l.init_cnn)

In [ ]:
VGG(arch=((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))).layer_summary(
    (1, 1, 224, 224))

## Training

- **VGG-11 is computationally more demanding than AlexNet we construct a network with a smaller number of channels.**
- more than sufficient for training on Fashion-MNIST. 
- **model training** process is similar to that of AlexNet in `sec_alexnet`. 
- observe the close match between validation and training loss, suggesting only a small amount of overfitting.


In [ ]:
model = VGG(arch=((1, 16), (1, 32), (2, 64), (2, 128), (2, 128)), lr=0.01)
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
data = d2l.FashionMNIST(batch_size=128, resize=(224, 224))
model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
# trainer.fit(model, data)

## Summary

- might argue that VGG is the first truly modern convolutional neural network. 
- VGG introduced key properties such as blocks of multiple convolutions and a preference for deep and narrow networks. 
- first network that is actually an entire family of similarly parametrized models, giving the practitioner ample trade-off between complexity and speed. 

## Exercises


1. Compared with AlexNet, VGG is much slower in terms of computation, and it also needs more GPU memory. 
    1. Compare the number of parameters needed for AlexNet and VGG.
    1. Compare the number of floating point operations used in the convolutional layers and in the fully connected layers. 
    1. How could you reduce the computational cost created by the fully connected layers?
1. When displaying the dimensions associated with the various layers of the network, we only see the information associated with 8 blocks (plus some auxiliary transforms), even though the network has 11 layers. Where did the remaining 3 layers go?
1. Use Table 1 in the VGG paper :cite:`Simonyan.Zisserman.2014` to construct other common models, such as VGG-16 or VGG-19.
1. Upsampling the resolution in Fashion-MNIST by a factor of $8$ from $28 \times 28$ to $224 \times 224$ dimensions is very wasteful. Try modifying the network architecture and resolution conversion, e.g., to 56 or to 84 dimensions for its input instead. Can you do so without reducing the accuracy of the network? Consider the VGG paper :cite:`Simonyan.Zisserman.2014` for ideas on adding more nonlinearities prior to downsampling.


## Network in Network (NiN)

- LeNet, AlexNet, and VGG share a common pattern: extract features via convolutions and pooling, then process via fully connected layers.
- This design has two major challenges:
  - Fully connected layers consume enormous memory (e.g., VGG-11 needs 400MB just for one layer)
  - Adding nonlinearity through fully connected layers would destroy spatial structure

- *Network in Network* (*NiN*) :cite:`Lin.Chen.Yan.2013` solves both problems by:
  - Using $1 \times 1$ convolutions for local nonlinearities across channels
  - Using global average pooling instead of fully connected layers


In [ ]:
from d2l import torch as d2l
import torch
from torch import nn

## NiN Blocks

- NiN applies a fully connected layer at each pixel location using $1 \times 1$ convolutions
- This differs from VGG by following the initial convolution with $1 \times 1$ convolutions instead of $3 \times 3$ convolutions
- Eliminates the need for large fully connected layers at the end

![Comparing the architectures of VGG and NiN, and of their blocks.](../img/nin.svg){centered=true width=40%}

---


In [ ]:
def nin_block(out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.LazyConv2d(out_channels, kernel_size, strides, padding), nn.ReLU(),
        nn.LazyConv2d(out_channels, kernel_size=1), nn.ReLU(),
        nn.LazyConv2d(out_channels, kernel_size=1), nn.ReLU())

## NiN Model

- NiN uses the same initial convolution sizes as AlexNet ($11\times 11$, $5\times 5$, and $3\times 3$)
- Instead of fully connected layers, NiN uses:
  - A NiN block with output channels equal to number of classes
  - A global average pooling layer
  - This design significantly reduces parameters


In [ ]:
class NiN(d2l.Classifier):
    def __init__(self, lr=0.1, num_classes=10):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.Sequential(
            nin_block(96, kernel_size=11, strides=4, padding=0),
            nn.MaxPool2d(3, stride=2),
            nin_block(256, kernel_size=5, strides=1, padding=2),
            nn.MaxPool2d(3, stride=2),
            nin_block(384, kernel_size=3, strides=1, padding=1),
            nn.MaxPool2d(3, stride=2),
            nn.Dropout(0.5),
            nin_block(num_classes, kernel_size=3, strides=1, padding=1),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten())
        self.net.apply(d2l.init_cnn)        

---

-  create a data example to see **the output shape of each block**


In [ ]:
NiN().layer_summary((1, 1, 224, 224))

## Training

- use Fashion-MNIST to train the model 


In [ ]:
model = NiN(lr=0.05)
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
data = d2l.FashionMNIST(batch_size=128, resize=(224, 224))
model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
# trainer.fit(model, data)

## Summary

- NiN has dramatically fewer parameters than AlexNet and VGG. 
- stems primarily from the fact that it needs no giant fully connected layers. 
- Instead, uses global average pooling to aggregate across all image locations after the last stage of the network body. 
- obviates the need for expensive (learned) reduction operations and replaces them by a simple average. 
- averaging operation does not harm accuracy. 
- averaging across a low-resolution representation (with many channels) also adds to the amount of translation invariance that the network can handle. 
- choosing fewer convolutions with wide kernels and replacing them by $1 \times 1$ convolutions aids the quest for fewer parameters further. 
- affords for a significant amount of nonlinearity across channels within any given location. 
- Both $1 \times 1$ convolutions and global average pooling significantly influenced subsequent CNN designs. 

## Exercises

1. Why are there two $1\times 1$ convolutional layers per NiN block? Increase their number to three. Reduce their number to one. What changes?
1. What changes if you replace the $1 \times 1$ convolutions by $3 \times 3$ convolutions? 
1. What happens if you replace the global average pooling by a fully connected layer (speed, accuracy, number of parameters)?
1. Calculate the resource usage for NiN.
    1. What is the number of parameters?
    1. What is the amount of computation?
    1. What is the amount of memory needed during training?
    1. What is the amount of memory needed during prediction?
1. What are possible problems with reducing the $384 \times 5 \times 5$ representation to a $10 \times 5 \times 5$ representation in one step?
1. Use the structural design decisions in VGG that led to VGG-11, VGG-16, and VGG-19 to design a family of NiN-like networks.


## Batch Normalization

- Training deep neural networks is difficult and slow to converge
- *Batch normalization* :cite:`Ioffe.Szegedy.2015` helps by:
  - Accelerating network convergence
  - Enabling training of very deep networks (100+ layers)
  - Providing inherent regularization


In [ ]:
from d2l import torch as d2l
import torch
from torch import nn

## Training Deep Networks

- When working with data, we often preprocess before training. 
- Choices regarding data preprocessing often make an enormous difference in the final results. 
- Recall our application of MLPs to predicting house prices (:numref:`sec_kaggle_house`). 
- first step when working with real data: standardize our input features to have zero mean $\boldsymbol{\mu} = 0$ and unit variance $\boldsymbol{\Sigma} = \boldsymbol{1}$  cross multiple observations :cite:`friedman1987exploratory`.
- one frequently rescales it such that the diagonal is unity, i.e., $\Sigma_ ii} = 1$.
- another strategy is to rescale vectors to unit length, possibly zero mean *per observation*. 
- can work well, e.g., for spatial sensor data. These preprocessing techniques and many more are beneficial to keep the estimation problem well controlled. 
- standardizing vectors also has the nice side-effect of constraining the function complexity of functions that act upon it. 
- Intuitively, this standardization plays nicely with our optimizers since it puts the parameters *a priori* at a similar scale. 

---


- natural to ask whether a corresponding normalization step *inside* a deep network might not be beneficial
- useful way of understanding it and its cousin, layer normalization :cite:`Ba.Kiros.Hinton.2016` within a unified framework. 
- Second, for a typical MLP or CNN, as we train, the variables  in intermediate layers (e.g., affine transformation outputs in MLP) may take values with widely varying magnitudes: 
  - both along the layers from input to output, across units in the same layer, and over time due to our updates to the model parameters.
  - inventors of batch normalization postulated informally that this drift in the distribution of such variables could hamper the convergence of the network. 
- Intuitively, we might conjecture that if one layer has variable activations that are 100 times that of another layer, this might necessitate compensatory adjustments in the learning rates. 
- Adaptive solvers such as AdaGrad :cite:`Duchi.Hazan.Singer.2011`, Adam :cite:`Kingma.Ba.2014`, Yogi :cite:`Zaheer.Reddi.Sachan.ea.2018`, or Distributed Shampoo :cite:`anil2020scalable` aim to address this from the viewpoint of optimization, e.g., by adding aspects of second-order methods. 

---


- alternative is to prevent the problem from occurring, simply by adaptive normalization.
- deeper networks are complex --> overfitting. 
- regularization becomes more critical. A common technique for regularization is noise injection. 
- has been known for a long time, e.g., with regard to noise injection for the inputs :cite:`Bishop.1995`. It also forms the basis of dropout in :numref:`sec_dropout`. 
- batch normalization conveys all three benefits: preprocessing, numerical stability, and regularization. 
- Batch normalization is applied to individual layers, or optionally, to all of them: 
- In each training iteration, we first normalize the inputs (of batch normalization) by subtracting their mean and dividing by their standard deviation, where both are estimated based on the statistics of the current minibatch. 
- Next, we apply a scale coefficient and an offset to recover the lost degrees of freedom. It is precisely due to this *normalization* based on *batch* statistics that *batch normalization* derives its name.

---


- Note that if we tried to apply batch normalization with minibatches of size 1, we would not be able to learn anything. 
- That is because after subtracting the means, each hidden unit would take value 0. 
- As you might guess, since we are devoting a whole section to batch normalization, with large enough minibatches, the approach proves effective and stable.
- One takeaway here is that when applying batch normalization, the choice of batch size is even more significant than without batch normalization, or at least, suitable calibration is needed as we might adjust it.
- Denote by $\mathcal{B}$ a minibatch and let $\mathbf{x} \in \mathcal{B}$ be an input to  batch normalization ($\mathrm{BN}$). In this case the batch normalization is defined as  follows: 
- 
$$\mathrm{BN}(\mathbf{x}) = \boldsymbol{\gamma} \odot \frac{\mathbf{x} - \hat{\boldsymbol{\mu}}_\mathcal{B}}{\hat{\boldsymbol{\sigma}}_\mathcal{B}} + \boldsymbol{\beta}.$$
:eqlabel:`eq_batchnorm`

---


- In :eqref:`eq_batchnorm`, $\hat{\boldsymbol{\mu}}_\mathcal{B}$ is the  sample mean and $\hat{\boldsymbol{\sigma}}_\mathcal{B}$ is the sample standard deviation of the minibatch $\mathcal{B}$.
- After applying standardization, the resulting minibatch has zero mean and unit variance. The choice of unit variance (vs. some other magic number) is an arbitrary choice. We recover this degree of freedom by including an elementwise *scale parameter* $\boldsymbol{\gamma}$  and *shift parameter* $\boldsymbol{\beta}$ that have the same shape as $\mathbf{x}$. Both are parameters that need to be learned as part of model training.
- The variable magnitudes for intermediate layers cannot diverge during training since batch normalization actively centers and rescales them back to a given mean and size (via $\hat \boldsymbol{\mu}}_\mathcal{B}$ and ${\hat{\boldsymbol{\sigma}}_\mathcal{B}}$). 
- Practical experience confirms that, as alluded to when discussing feature rescaling, batch normalization seems to allow for more aggressive learning rates. 


---


- We calculate $\hat{\boldsymbol{\mu}}_\mathcal{B}$ and ${\hat{\boldsymbol{\sigma}}_\mathcal{B}}$ in :eqref:`eq_batchnorm` as follows:

$$\hat{\boldsymbol{\mu}}_\mathcal{B} = \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} \mathbf{x}
\text{ and }
\hat{\boldsymbol{\sigma}}_\mathcal{B}^2 = \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} (\mathbf{x} - \hat{\boldsymbol{\mu}}_{\mathcal{B}})^2 + \epsilon.$$

- Note that we add a small constant $\epsilon > 0$ to the variance estimate to ensure that we never attempt division by zero, even in cases where the empirical variance estimate might be very small or even vanish.
- The estimates $\hat{\boldsymbol{\mu}}_\mathcal{B}$ and ${\hat{\boldsymbol{\sigma}}_\mathcal{B}}$ counteract the scaling issue by using noisy estimates of mean and variance. 
- You might think that this noisiness should be a problem. Quite to the contrary, this is actually beneficial.


---


- This turns out to be a recurring theme in deep learning. For reasons that are not yet well-characterized theoretically, various sources of noise in optimization often lead to faster training and less overfitting:
- this variation appears to act as a form of regularization. :citet:`Teye.Azizpour.Smith.2018` and :citet:`Luo.Wang.Shao.ea.2018` related the properties of batch normalization to Bayesian priors and penalties, respectively. 
- In particular, this sheds some light on the puzzle of why batch normalization works best for moderate minibatches sizes in the $50 \sim 100$ range.
- This particular size of minibatch seems to inject just the "right amount" of noise per layer, both in terms of scale via $\hat{\boldsymbol{\sigma}}$, and in terms of offset via $\hat{\boldsymbol{\mu}}$: 
- a larger minibatch regularizes less due to the more stable estimates, whereas tiny minibatches destroy useful signal due to high variance. 


---


- Exploring this direction further, considering alternative types of preprocessing and filtering may yet lead to other effective types of regularization.
- Fixing a trained model, you might think that we would prefer using the entire dataset to estimate the mean and variance. 
- Once training is complete, why would we want the same image to be classified differently, depending on the batch in which it happens to reside? 
- During training, such exact calculation is infeasible because the intermediate variables for all data examples change every time we update our model. 
- However, once the model is trained, we can calculate the means and variances of each layer's variables based on the entire dataset.
- Indeed this is standard practice for models employing batch normalization and thus batch normalization layers function differently in *training mode* (normalizing by minibatch statistics) and in *prediction mode* (normalizing by dataset statistics). 
- In this form they closely resemble the behavior of dropout regularization of :numref:`sec_dropout`, where noise is only injected during training.

## Batch Normalization Layers

- Batch normalization implementations for fully connected layers and convolutional layers are slightly different.
- One key difference between batch normalization and other layers is that because batch normalization operates on a full minibatch at a time, we cannot just ignore the batch dimension as we did before when introducing other layers.

## Fully Connected Layers

- When applying batch normalization to fully connected layers, the original paper inserted batch normalization after the affine transformation and *before* the nonlinear activation function. Later applications experimented with inserting batch normalization right *after* activation functions :cite:`Ioffe.Szegedy.2015`.
- Denoting the input to the fully connected layer by $\mathbf{x}$, the affine transformation by $\mathbf{W}\mathbf{x} + \mathbf{b}$ (with the weight parameter $\mathbf{W}$ and the bias parameter $\mathbf{b}$), and the activation function by $\phi$, we can express the computation of a batch-normalization-enabled, fully connected layer output $\mathbf{h}$ as follows:

$$\mathbf{h} = \phi(\mathrm{BN}(\mathbf{W}\mathbf{x} + \mathbf{b}) ).$$

- Recall that mean and variance are computed on the *same* minibatch on which the  ransformation is applied.

## Convolutional Layers

- Similarly, with convolutional layers, we can apply batch normalization after the convolution and before the nonlinear activation function. The key difference from batch normalization in fully connected layers is that we apply the operation on a per-channel basis *across all locations*. 
- This is compatible with our assumption of translation invariance that led to convolutions: we assumed that the specific location of a pattern within an image was not critical for the purpose of understanding.
- Assume that our minibatches contain $m$ examples and that for each channel, the output of the convolution has height $p$ and width $q$.
- For convolutional layers, we carry out each batch normalization over the $m \cdot p \cdot q$ elements per output channel simultaneously.
- Thus, we collect the values over all spatial locations when computing the mean and variance and consequently apply the same mean and variance within a given channel to normalize the value at each spatial location.
- Each channel has its own scale and shift parameters, both of which are scalars.

## Layer Normalization
- Note that in the context of convolutions the batch normalization is well-defined even for minibatches of size 1: after all, we have all the locations across an image to average
- consequently, mean and variance are well defined, even if it is just within a single observation. 
- This consideration led :citet:`Ba.Kiros.Hinton.2016` to introduce the notion of *layer normalization*. 
- It works just like a batch norm, only that it is applied to one observation at a time. Consequently both the offset and the scaling factor are scalars. Given an $n$-dimensional vector $\mathbf{x}$ layer norms are given by 

$$\mathbf{x} \rightarrow \mathrm{LN}(\mathbf{x}) =  \frac{\mathbf{x} - \hat{\mu}}{\hat\sigma},$$

where scaling and offset are applied coefficient-wise
and given by 

$$\hat{\mu} \stackrel{\mathrm{def}}{=} \frac{1}{n} \sum_{i=1}^n x_i \text{ and }
\hat{\sigma}^2 \stackrel{\mathrm{def}}{=} \frac{1}{n} \sum_{i=1}^n (x_i - \hat{\mu})^2 + \epsilon.$$

---

- As before we add a small offset $\epsilon > 0$ to prevent division by zero. 
- One of the major benefits of using layer normalization is that it prevents divergence.
- After all, ignoring $\epsilon$, the output of the layer normalization is scale independent. 
- That is, we have $\mathrm{LN}(\mathbf{x}) \approx \mathrm{LN}(\alpha \mathbf{x})$ for any choice of $\alpha \neq 0$. 
- This becomes an equality for $|\alpha| \to \infty$ (the approximate equality is due to the offset $\epsilon$ for the variance).

---


- Another advantage of the layer normalization is that it does not depend on the minibatch size. 
- It is also independent of whether we are in training or test regime. 
- In other words, it is simply a deterministic transformation that standardizes the activations to a given scale. 
- This can be very beneficial in preventing divergence in optimization. We skip further details and recommend the interested reader to consult the original paper.

## Batch Normalization During Prediction

- mentioned earlier, batch normalization typically behaves differently in training mode and prediction mode. 
- First, the noise in the sample mean and the sample variance arising from estimating each on minibatches are no longer desirable once we have trained the model. 
- Second, we might not have the luxury of computing per-batch normalization statistics. 
- For example, we might need to apply our model to make one prediction at a time.
- Typically, after training, we use the entire dataset to compute stable estimates of the variable statistics and then fix them at prediction time. 
- Consequently, batch normalization behaves differently during training and at test time. Recall that dropout also exhibits this characteristic.


## Discussion

- Intuitively, batch normalization is thought to make the optimization landscape smoother. 
- However, we must be careful to distinguish between speculative intuitions and true explanations for the phenomena that we observe when training deep models. 
- Recall that we do not even know why simpler deep neural networks (MLPs and conventional CNNs) generalize well in the first place. 
- Even with dropout and weight decay, they remain so flexible that their ability to generalize to unseen data likely needs significantly more refined learning-theoretic generalization guarantees.

---

- On a more practical note, there are a number of aspects worth remembering about batch normalization: 
* During model training, batch normalization continuously adjusts the intermediate output of
  the network by utilizing the mean and standard deviation of the minibatch, so that the
  values of the intermediate output in each layer throughout the neural network are more stable.
* Batch normalization for fully connected layers and convolutional layers are slightly different. In fact,
  for convolutional layers, layer normalization can sometimes be used as an alternative.
* Like a dropout layer, batch normalization layers have different behaviors
  in training mode and prediction mode.
* Batch normalization is useful for regularization and improving convergence in optimization. On the other hand,
  the original motivation of reducing internal covariate shift seems not to be a valid explanation.
* For more robust models that are less sensitive to input perturbations, consider removing batch normalization :cite:`wang2022removing`.

## Densely Connected Networks (DenseNet)

- *DenseNet* (dense convolutional network) is to some extent the logical extension of this  cite:`Huang.Liu.Van-Der-Maaten.ea.2017`. 
- DenseNet is characterized by both the connectivity pattern where each layer connects to all the preceding layers and the concatenation operation (rather than the addition operator in ResNet) to preserve and reuse features from earlier layers. 
- To understand how to arrive at it, let's take a small detour to mathematics.


In [ ]:
from d2l import torch as d2l
import torch
from torch import nn

## From ResNet to DenseNet

- Recall the Taylor expansion for functions. For the point $x = 0$ it can be written as

$$f(x) = f(0) + x \cdot \left[f'(0) + x \cdot \left[\frac{f''(0)}{2!}  + x \cdot \left[\frac{f'''(0)}{3!}  + \ldots \right]\right]\right].$$

- The key point is that it decomposes a function into increasingly higher order terms. In a similar vein, ResNet decomposes functions into

$$f(\mathbf{x}) = \mathbf{x} + g(\mathbf{x}).$$

- That is, ResNet decomposes $f$ into a simple linear term and a more complex nonlinear one. 
- What if we wanted to capture (not necessarily add) information beyond two terms? One such solution is DenseNet :cite:`Huang.Liu.Van-Der-Maaten.ea.2017`.

---

![The main difference between ResNet (left) and DenseNet (right) in cross-layer connections: use of addition and use of concatenation. ](../img/densenet-block.svg){centered=true width=40%}

- As shown in `fig_densenet_block`, the key difference between ResNet and DenseNet is that in the latter case outputs are *concatenated* (denoted by $[,]$) rather than added.
- As a result, we perform a mapping from $\mathbf{x}$ to its values after applying an increasingly complex sequence of functions:

$$\mathbf{x} \to \left[
\mathbf{x},
f_1(\mathbf{x}),
f_2\left(\left[\mathbf{x}, f_1\left(\mathbf{x}\right)\right]\right), f_3\left(\left[\mathbf{x}, f_1\left(\mathbf{x}\right), f_2\left(\left[\mathbf{x}, f_1\left(\mathbf{x}\right)\right]\right)\right]\right), \ldots\right].$$

---

- all these functions are combined in MLP to reduce the number of features again. In terms of implementation this is quite simple: 
- rather than adding terms, we concatenate them. 
- name DenseNet arises from the fact that the dependency graph between variables becomes quite dense. 
- last layer of such a chain is densely connected to all previous layers. The dense connections are shown in `fig_densenet`.

![Dense connections in DenseNet. Note how the dimensionality increases with depth.](../img/densenet.svg){centered=true width=40%}


---

- main components that compose a DenseNet are *dense blocks* and *transition layers*. The former define how the inputs and outputs are concatenated, while the latter control the number of channels so that it is not too large,  since the expansion $\mathbf{x} \to \left[\mathbf{x}, f_1(\mathbf{x}), f_2\left(\left[\mathbf{x}, f_1\left(\mathbf{x}\right)\right]\right), \ldots \right]$ can be quite high-dimensional.

## [**Dense Blocks**]

- DenseNet uses the modified "batch normalization, activation, and convolution" structure of ResNet (see the exercise in :numref:`sec_resnet`). First, we implement this convolution block structure.


In [ ]:
def conv_block(num_channels):
    return nn.Sequential(
        nn.LazyBatchNorm2d(), nn.ReLU(),
        nn.LazyConv2d(num_channels, kernel_size=3, padding=1))

- *dense block* consists of multiple convolution blocks, each using the same number of output channels. In the forward propagation, however, we concatenate the input and output of each convolution block on the channel dimension. Lazy evaluation allows us to adjust the dimensionality automatically.


In [ ]:
class DenseBlock(nn.Module):
    def __init__(self, num_convs, num_channels):
        super(DenseBlock, self).__init__()
        layer = []
        for i in range(num_convs):
            layer.append(conv_block(num_channels))
        self.net = nn.Sequential(*layer)

    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            ## Concatenate input and output of each block along the channels
            X = torch.cat((X, Y), dim=1)
        return X

---

- In the following example, we [**define a `DenseBlock` instance**] with 2 convolution blocks of 10 output channels. 
- When using an input with 3 channels, we will get an output with  $3 + 10 + 10=23$ channels. The number of convolution block channels controls the growth in the number of output channels relative to the number of input channels. This is also referred to as the *growth rate*.


In [ ]:
blk = DenseBlock(2, 10)
X = torch.randn(4, 3, 8, 8)
Y = blk(X)
Y.shape

## [**Transition Layers**]

- Since each dense block will increase the number of channels, adding too many of them will lead to an excessively complex model. A *transition layer* is used to control the complexity of the model. It reduces the number of channels by using an $1\times 1$ convolution. Moreover, it halves the height and width via average pooling with a stride of 2.


In [ ]:
def transition_block(num_channels):
    return nn.Sequential(
        nn.LazyBatchNorm2d(), nn.ReLU(),
        nn.LazyConv2d(num_channels, kernel_size=1),
        nn.AvgPool2d(kernel_size=2, stride=2))

---

- [**Apply a transition layer**] with 10 channels to the output of the dense block in the previous example.  This reduces the number of output channels to 10, and halves the height and width.


In [ ]:
blk = transition_block(10)
blk(Y).shape

## DenseNet Model

- Next, we will construct a DenseNet model. DenseNet first uses the same single convolutional layer and max-pooling layer as in ResNet.


In [ ]:
class DenseNet(d2l.Classifier):
    def b1(self):
        return nn.Sequential(
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            nn.LazyBatchNorm2d(), nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

---

- Then, similar to the four modules made up of residual blocks that ResNet uses, DenseNet uses four dense blocks. 
- Similar to ResNet, we can set the number of convolutional layers used in each dense block. Here, we set it to 4, consistent with the ResNet-18 model in :numref:`sec_resnet`. 
- Furthermore, we set the number of channels (i.e., growth rate) for the convolutional layers in the dense block to 32, so 128 channels will be added to each dense block. 
- Similar to ResNet, a global pooling layer and a fully connected layer are connected at the end to produce the output.

---


In [ ]:
@d2l.add_to_class(DenseNet)
def __init__(self, num_channels=64, growth_rate=32, arch=(4, 4, 4, 4),
             lr=0.1, num_classes=10):
    super(DenseNet, self).__init__()
    self.save_hyperparameters()


    self.net = nn.Sequential(self.b1())
    for i, num_convs in enumerate(arch):
        self.net.add_module(f'dense_blk{i+1}', DenseBlock(num_convs,
                                                            growth_rate))
        ## The number of output channels in the previous dense block
        num_channels += num_convs * growth_rate
        ## A transition layer that halves the number of channels is added
        ## between the dense blocks
        if i != len(arch) - 1:
            num_channels //= 2
            self.net.add_module(f'tran_blk{i+1}', transition_block(
                num_channels))
    self.net.add_module('last', nn.Sequential(
        nn.LazyBatchNorm2d(), nn.ReLU(),
        nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(),
        nn.LazyLinear(num_classes)))
    self.net.apply(d2l.init_cnn)

## [**Training**]

Since we are using a deeper network here, in this section, we will reduce the input height and width from 224 to 96 to simplify the computation.


In [ ]:
model = DenseNet(lr=0.01)
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
data = d2l.FashionMNIST(batch_size=128, resize=(96, 96))
# trainer.fit(model, data)

## Summary and Discussion

- main components that compose DenseNet are dense blocks and transition layers. For the latter, we need to keep the dimensionality under control when composing the network by adding transition layers that shrink the number of channels again. 
- In terms of cross-layer connections, unlike ResNet, where inputs and outputs are added together, DenseNet concatenates inputs and outputs on the channel dimension. 
- concat --> heavy GPU memory consumption. 

## Exercises

1. Why do we use average pooling rather than max-pooling in the transition layer?
1. One of the advantages mentioned in the DenseNet paper is that its model parameters are smaller than those of ResNet. Why is this the case?
1. One problem for which DenseNet has been criticized is its high memory consumption.
    1. Is this really the case? Try to change the input shape to $224\times 224$ to see the actual GPU memory consumption empirically.
    1. Can you think of an alternative means of reducing the memory consumption? How would you need to change the framework?
1. Implement the various DenseNet versions presented in Table 1 of the DenseNet paper :cite:`Huang.Liu.Van-Der-Maaten.ea.2017`.
1. Design an MLP-based model by applying the DenseNet idea. Apply it to the housing price prediction task in :numref:`sec_kaggle_house`.


## U-Net Architecture

- U-Net was originally developed for biomedical image segmentation :cite:`ronneberger2015u`
- Key features:
  - Symmetric encoder-decoder structure
  - Skip connections between corresponding encoder-decoder levels
  - Designed to work with fewer training images
  - Preserves spatial information through skip connections

![U-Net architecture showing the contracting path (left) and expansive path (right). Image adapted from :cite:`ronneberger2015u`.](../img/u-net-architecture.png){centered=true width=40%}

## U-Net Components

- **Contracting Path (Encoder)**:
  - Series of convolutional and max pooling operations
  - Each step doubles the number of feature channels
  - Captures context and reduces spatial dimensions

- **Expansive Path (Decoder)**:
  - Upsampling of feature maps
  - Concatenation with corresponding encoder features
  - Successive convolutions to combine features
  - Gradually recovers spatial resolution

## U-Net Skip Connections

- Skip connections are crucial for U-Net's success:
  - Connect encoder layers to corresponding decoder layers
  - Allow fine-grained details to flow to decoder
  - Help prevent vanishing gradients
  - Enable precise localization combined with context


In [ ]:
class UNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        return x

## U-Net Applications

- Originally designed for medical image segmentation
- Now widely used in many domains:
  - Object detection
  - Industrial defect detection
  - General segmentation tasks

- Key advantages:
  - Works well with limited training data
  - Precise localization
  - End-to-end training
  - Fast inference
